In [1]:
from sqlalchemy import create_engine
# Correctly creating an SQLAlchemy engine for SQLite
engine = create_engine('sqlite:///datam.db')

# Connect to the database (this is where SQLAlchemy takes over)
connection = engine.connect()

In [ ]:
from sqlalchemy import MetaData
metadata = MetaData()
from sqlalchemy import Table, Column, Integer, Numeric, String, Boolean,select,DateTime
from dateutil.relativedelta import relativedelta
from datetime import datetime
users = Table('Users', metadata,
Column('username', String(50),unique=True),
Column('password', String(255)),
Column('description', String(55)),
Column('admin', Boolean()),
Column('user_id', Integer() ,primary_key=True, autoincrement=True, unique=True, index=True,nullable=False))

In [ ]:
students=Table('Students',metadata,
Column('id', Integer() ,primary_key=True, autoincrement=True, unique=True, index=True,nullable=False),
Column('name', String(50)),
Column('subject1', Numeric()),
Column('subject2', Numeric()),
Column('subject3', Numeric()),
Column('subject4', Numeric()),
Column('subject5', Numeric()),
Column('subject6', Numeric()),
Column('breakfast', Numeric()),
Column('lunch', Numeric()),
Column('snacks', Numeric()),
Column('dinner', Numeric()),
)

In [ ]:
issues=Table('Issue',metadata,
Column('id', Integer() ,primary_key=True, autoincrement=True, unique=True, index=True,nullable=False),
Column('issue', String(50)),
Column ('due',DateTime(),default=datetime.now()+relativedelta(months=1)),
Column('student_id',Integer()))


In [ ]:
from sqlalchemy import Enum


menu=Table('Menu',metadata,
           Column('id', Integer() ,primary_key=True, autoincrement=True, unique=True, index=True,nullable=False),
           Column('name', String(50),unique=True),
           Column('meal',Enum('breakfast', 'lunch', 'dinner', 'snacks')))

In [ ]:
from sqlalchemy import ForeignKeyConstraint
votes = Table('Votes', metadata,
    Column('id', Integer(), primary_key=True, autoincrement=True,unique=True),
    Column('student_id', Integer(), unique=True),  # Each student can only vote once
    Column('breakfast', Integer()),
    Column('lunch', Integer()),  # Fixed typo (was 'lunch' in your code)
    Column('dinner', Integer()),
    Column('snacks', Integer()),
    ForeignKeyConstraint(['student_id'], ['Students.id']),
    ForeignKeyConstraint(['breakfast'], ['Menu.id']),
    ForeignKeyConstraint(['lunch'], ['Menu.id']),
    ForeignKeyConstraint(['dinner'], ['Menu.id']),
    ForeignKeyConstraint(['snacks'], ['Menu.id'])
)

In [ ]:
from sqlalchemy import ForeignKeyConstraint
issues.append_constraint(
    ForeignKeyConstraint(['student_id'], ['Students.id'])
)

In [4]:
# Create the table in the database
metadata.create_all(engine)

# Connect to the database
connection = engine.connect()
print("Database connected and table created (if not exists).")

Database connected and table created (if not exists).


In [5]:
import bcrypt
def hash_pass(password):
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt()).decode("utf-8")

In [6]:
from sqlalchemy.exc import SQLAlchemyError

def insert_user(username, password, description="", admin=False):
    try:
        ins = users.insert().values(
            username=username,
            password=hash_pass(password),
            admin=admin,
            description=description
        )

        with engine.connect() as conn:
            result = conn.execute(ins)
            conn.commit()  # Ensure the changes are saved 
            return True,f"User {username} inserted successfully. ID: {result.inserted_primary_key}"

    except SQLAlchemyError as e:  # Catch database-related errors
        print( f"Error inserting user: {str(e)}")
        return False,e

    except Exception as e:  # Catch other unexpected errors
        print( f"Unexpected error: {str(e)}")
        return False,e
    

In [ ]:
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.sql import select
from bcrypt import checkpw

def login_user(username, password):
    try:
        query = select(users).where(users.c.username == username)

        with engine.connect() as conn:
            result = conn.execute(query)
            user = result.fetchone()  # Fetch one row

        if user is None:
            return False, "User not found"

        stored_hashed_password = user.password

        # Compare provided password with hashed password
        if checkpw(password.encode('utf-8'), stored_hashed_password.encode('utf-8')):
            return True,user.admin, "Login successful"
        else:
            return False, "Incorrect password"

    except SQLAlchemyError as e:
        print(f"Error logging in user: {str(e)}")
        return False, e

    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return False, e

In [ ]:
def get_student(id):
    try:
        query=select(students).where(students.c.id==id)
        with engine.connect() as conn:
                result = conn.execute(query)
                user = result.fetchone()  # Fetch one row
                if user is None:
                    return False, "User not found"
                else:
                     return True,user
    except SQLAlchemyError as e:
        print(f"Error logging in user: {str(e)}")
        return False, e

    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return False, e
          
    

In [ ]:
#inserting into student
def insert_student(name="",subject1=50,subject2=50,subject3=50,subject4=50,subject5=50,subject6=50,breakfast=0,lunch=0,snacks=0,dinner=0):
        try:
                ins = students.insert().values(
                        name=name,
                        subject1 = subject1,
                        subject2 = subject2,
                        subject3 = subject3,
                        subject4 = subject4,
                        subject5 = subject5,
                        subject6 = subject6,
                        breakfast = breakfast,
                        lunch = lunch,
                        snacks = snacks,
                        dinner = dinner,
                                )
                with engine.connect() as conn:
                        result = conn.execute(ins)
                        conn.commit()  # Ensure the changes are saved 
                        return True,f"student  inserted successfully. ID: {result.inserted_primary_key}" 

        except SQLAlchemyError as e:  # Catch database-related errors
                print( f"Error inserting user: {str(e)}")
                return  False,e
        except Exception as e:  # Catch other unexpected errors
                print( f"Unexpected error: {str(e)}")
                return  False,e

True User  inserted successfully. ID: (2,)


In [ ]:
#inserting into student
from sqlalchemy import update
def update_student(id,name="",subject1=50,subject2=50,subject3=50,subject4=50,subject5=50,subject6=50,breakfast=0,lunch=0,snacks=0,dinner=0):
        try: 
                result=get_student(id)
                if result[0]:
                        ins = students.update().where(students.c.id==id).values(
                                name=name,
                                subject1 = subject1,
                                subject2 = subject2,
                                subject3 = subject3,
                                subject4 = subject4,
                                subject5 = subject5,
                                subject6 = subject6,
                                breakfast = breakfast,
                                lunch = lunch,
                                snacks = snacks,
                                dinner = dinner,
                                        )
                        with engine.connect() as conn:
                                result = conn.execute(ins)
                                conn.commit()  # Ensure the changes are saved 
                                return True,f"student  updated successfully. ID: {id}" 
                else:
                        return False,f"student didn't existed for this  ID: {id}"


        except SQLAlchemyError as e:  # Catch database-related errors
                print( f"Error inserting user: {str(e)}")
                return  False,e
        except Exception as e:  # Catch other unexpected errors
                print( f"Unexpected error: {str(e)}")
                return  False,e

In [ ]:
from sqlalchemy import delete
def delete_student(id):
    try:
        result=get_student(id)
        if result[0]:
                query=students.delete().where(students.c.id==id)
                with engine.connect() as conn:
                        result = conn.execute(query)
                        conn.commit()  # Fetch one row
                        if result is None:
                            return False, "deletion not happpend"
                        else:
                            return True,f"{id} deleted"
        else:
             return False,f"student didn't existed for this  ID: {id}"
            
    except SQLAlchemyError as e:
        print(f"Error logging in user: {str(e)}")
        return False, e

    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return False, e
          
    

In [ ]:
def get_all_students(limit=10,page=0):
    try:
        offset_val=page*limit
        query=select(students).offset(offset_val).limit(limit+1)
        with engine.connect() as conn:
                result = conn.execute(query)
                user = result.fetchall()  # Fetch one row
                if user is None:
                    return False, "User not found"
                else:
                     return True,user
    except SQLAlchemyError as e:
        print(f"Error logging in user: {str(e)}")
        return False, e

    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return False, e

In [ ]:
def get_student_issue(limit=10,page=0,id=id):
    try:
        offset_val=page*limit
        query=select(issues).where(issues.c.student_id==id).offset(offset_val).limit(limit+1)
        with engine.connect() as conn:
                result = conn.execute(query)
                user = result.fetchall()  # Fetch one row
                if user is None:
                    return False, "issues not found"
                else:
                     return True,user
    except SQLAlchemyError as e:
        print(f"Error retrieving in user: {str(e)}")
        return False, e

    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return False, e

In [ ]:
def get_all_issues():
    with engine.connect() as connection:
        query=select(issues)
        result=connection.execute(query)
        #print(result.fetchall())
        return result.fetchall()

In [ ]:
def add_issue(id,issue):
    try:
        ins = issues.insert().values(
            issue=issue,
            student_id=id
        )

        with engine.connect() as conn:
            result = conn.execute(ins)
            conn.commit()  # Ensure the changes are saved 
            return True,f" issue  inserted successfully. issue  ID: {result.inserted_primary_key}"

    except SQLAlchemyError as e:  # Catch database-related errors
        print( f"Error inserting issue: {str(e)}")
        return False,e

    except Exception as e:  # Catch other unexpected errors
        print( f"Unexpected error: {str(e)}")
        return False,e
    

In [ ]:
def delete_issue(id):
    try :
        query =select(issues).where(issues.c.id==id)
        with engine.connect() as conn:
            result=conn.execute(query)
            if result.fetchone() is None:
                return False,"issue didn't exist"
            else:
                try:
                    ins = delete(issues).where(issues.c.id==id)

                    with engine.connect() as conn:
                        result = conn.execute(ins)
                        conn.commit()  # Ensure the changes are saved 
                        return True,"issue  deleted successfully"

                except SQLAlchemyError as e:  # Catch database-related errors
                    print( f"Error inserting issue: {str(e)}")
                    return False,e

                except Exception as e:  # Catch other unexpected errors
                    print( f"Unexpected error: {str(e)}")
                    return False,e
                
                
    except SQLAlchemyError as e:  # Catch database-related errors
        print( f"Error inserting issue: {str(e)}")
        return False,e

    except Exception as e:  # Catch other unexpected errors
        print( f"Unexpected error: {str(e)}")
        return False,e


In [ ]:
items_list = [
    # Breakfast items
    {"name": "Idly", "meal": "breakfast"},
    {"name": "Dosa", "meal": "breakfast"},
    {"name": "Poha", "meal": "breakfast"},
    {"name": "Upma", "meal": "breakfast"},
    
    # Lunch items
    {"name": "Dal Rice", "meal": "lunch"},
    {"name": "Vegetable Biryani", "meal": "lunch"},
    {"name": "Rajma Chawal", "meal": "lunch"},
    {"name": "Roti Sabzi", "meal": "lunch"},
    
    {"name": "Samosa", "meal": "snacks"},
    {"name": "Pakora", "meal": "snacks"},
    {"name": "Bhel Puri", "meal": "snacks"},
    {"name": "Cutlet", "meal": "snacks"},
    
    # Dinner items
    {"name": "Khichdi", "meal": "dinner"},
    {"name": "Chapati with Dal", "meal": "dinner"},
    {"name": "Vegetable Pulao", "meal": "dinner"},
    {"name": "Curd Rice", "meal": "dinner"}
]

In [ ]:
def create_menu():
    try:
        ins=menu.insert()
        with engine.connect() as conn:
            conn.execute(menu.delete())
            conn.commit()
            result=conn.execute(ins,items_list)
            conn.commit()
            if len(result.inserted_primary_key_rows)>0:
                return True, "menu created"
            else:
                return False,"error creating menu"
    except SQLAlchemyError as e:  # Catch database-related errors
        print( f"Error inserting issue: {str(e)}")
        return False,e

    except Exception as e:  # Catch other unexpected errors
        print( f"Unexpected error: {str(e)}")
        return False,e

In [ ]:
def get_menu():
    try:
        ins=select(menu)
        with engine.connect() as conn:
            result=conn.execute(ins)
            user=result.fetchall()
            if user:
                return True, user
            else:
                return False,"error showing menu"
    except SQLAlchemyError as e:  # Catch database-related errors
        print( f"Error inserting issue: {str(e)}")
        return False,e

    except Exception as e:  # Catch other unexpected errors
        print( f"Unexpected error: {str(e)}")
        return False,e


In [ ]:
def student_vote(student_id, breakfast, lunch, snacks, dinner):
    try:
        ins = votes.insert()
        with engine.connect() as conn:
            # First check if student already voted
            existing = conn.execute(
                votes.select().where(votes.c.student_id == student_id)
            ).fetchone()
            
            if existing:
                return False, "Student has already voted"
                
            result = conn.execute(ins, {
                'student_id': student_id,
                'breakfast': breakfast,
                'lunch': lunch,
                'snacks': snacks,
                'dinner': dinner
            })
            conn.commit()
            
            if result.inserted_primary_key:  # Changed from inserted_primary_key_rows
                print(f"New vote ID: {result.inserted_primary_key[0]}")
                return True, "Vote successfully cast"
            return False, "Failed to cast vote"
            
    except SQLAlchemyError as e:
        print(f"Database error: {str(e)}")
        return False, str(e)
    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return False, str(e)

In [ ]:
def delete_votes(id):
    try :
        query =votes.delete()
        with engine.connect() as conn:
            result=conn.execute(query)
            conn.commit()
            return True,"deleted successfully"  
    except SQLAlchemyError as e:  # Catch database-related errors
        print( f"Error inserting issue: {str(e)}")
        return False,e

    except Exception as e:  # Catch other unexpected errors
        print( f"Unexpected error: {str(e)}")
        return False,e

    

In [ ]:
from sqlalchemy import func
dic={"breakfast":1,"lunch":5,"snacks":9,"dinner":13}
def finalize_menu():
# Assuming you have a table called 'votes' with column 'breakfast'
    stmt = select(
        votes.c.breakfast,
        func.count(votes.c.breakfast).label('count')
    ).group_by(
        votes.c.breakfast
    ).order_by(
        func.count(votes.c.breakfast).desc()
    ).limit(1)

    with engine.connect() as conn:
        result = conn.execute(stmt).fetchone()
        if result:
            print(f"The most frequent breakfast is {result[0]} appearing {result[1]} times")
            dic['breakfast']=result[0]
        else:
            print("No data found")
        stmt = select(
        votes.c.lunch,
        func.count(votes.c.lunch).label('count')
    ).group_by(
        votes.c.lunch
    ).order_by(
        func.count(votes.c.lunch).desc()
    ).limit(1)

    with engine.connect() as conn:
        result = conn.execute(stmt).fetchone()
        if result:
            print(f"The most frequent lunch is {result[0]} appearing {result[1]} times")
            dic['lunch']=result[0]
        else:
            print("No data found")
        stmt = select(
        votes.c.snacks,
        func.count(votes.c.snacks).label('count')
    ).group_by(
        votes.c.snacks
    ).order_by(
        func.count(votes.c.snacks).desc()
    ).limit(1)

    with engine.connect() as conn:
        result = conn.execute(stmt).fetchone()
        if result:
            print(f"The most frequent snacks is {result[0]} appearing {result[1]} times")
            dic['snacks']=result[0]
        else:
            print("No data found")
        stmt = select(
        votes.c.dinner,
        func.count(votes.c.dinner).label('count')
    ).group_by(
        votes.c.dinner
    ).order_by(
        func.count(votes.c.dinner).desc()
    ).limit(1)

    with engine.connect() as conn:
        result = conn.execute(stmt).fetchone()
        if result:
            print(f"The most frequent dinner is {result[0]} appearing {result[1]} times")
            dic['dinner']=result[0]
        else:
            print("No data found")
    stmt = select(menu.c.name).where(
        menu.c.id.in_([dic['breakfast'], 
                    dic['lunch'], 
                    dic['snacks'], 
                    dic['dinner']])
    )

    # Execute the query
    with engine.connect() as conn:
        result = conn.execute(stmt)
        menu_names = [row[0] for row in result]
        return True,menu_names